# Plan
    # Overall objective: 
    Create an interactive dashboard for creators

    ## 1. Clean the data
        ### 1.1 Convert the data into the handiest format
        ### 1.2 Remove non-english comments using the tool of Google
        ### 1.3 Remove emojis and other unwanted elements (e.g., bots)
        ### 1.4 Remove stop words
    ## 2. Run basic statistics on the data
    ## 3. Start the network visualisation
        ## 3.1 Apply the topic detection and link prediction algorithm
        ## 3.2 Apply sentiment analysis to further label the comments
        ## 3.3 Visualise the output


# Import modules

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np
import re

# Import data

In [ ]:
# data = pd.read_json("Scrapped_Data.txt")
# data = pd.read_json("pornhub_videos_per_creator_TESTData.txt")
data = pd.read_json("pornhub_videos_per_creator_TRAINData.txt")

# The data is flipped around so we transpose it
data = data.T

# Remove rows with missing values
data = data.dropna(subset=["title"])
data = data.reset_index(drop=True)

In [ ]:
data.head()

,title,views,up_votes,down_votes,percent,author,author_subscriber,categories,tags,production,...,duration,upload_date,pornstars,download_urls,thumbnail_url,number_of_comment,comments,related_videos,url,error
0,Sunbathing Ended up with BJ & Hot Quickie. I H...,"9,177,422",28,5,85,Mini Diva,849,"[Babe, Big Ass, Big Tits, Blowjob, Exclusive, ...","[butt, big boobs, teenager, young, public, out...",homemade,...,691,2020-09-26T12:24:07.000Z,No Data,{'1080P': 'https://ev-h.phncdn.com/hls/videos/...,https://di.phncdn.com/videos/202009/26/3551703...,515,[{'avatar': 'https://ei.phncdn.com/pics/users/...,[{'title': 'Incredible First Meeting With Step...,https://www.pornhub.com/view_video.php?viewkey...,NaN
1,He Fucked me Hard in the Shower and came on my...,"2,559,483",11,3,79,Mini Diva,849,"[Amateur, Babe, Big Tits, Blowjob, Brunette, C...","[big boobs, teenager, young, rough, hard doggy...",homemade,...,663,2019-05-07T23:15:48.000Z,No Data,{'1080P': 'https://dv-h.phncdn.com/hls/videos/...,https://ei.phncdn.com/videos/201905/07/2224119...,193,[{'avatar': 'https://di.phncdn.com/pics/users/...,[{'title': 'Fuck me hard until I'll cum twice ...,https://www.pornhub.com/view_video.php?viewkey...,NaN
2,Lazy Weekend Sex and Loud Orgasms with my Favo...,"21,206,931",76,25,75,Mini Diva,849,"[Amateur, Babe, Big Ass, Big Tits, Blowjob, Cu...","[adult toys, masturbate, butt, big boobs, teen...",homemade,...,1068,2020-02-07T19:50:05.000Z,No Data,{'1080P': 'https://cv-h.phncdn.com/hls/videos/...,https://di.phncdn.com/videos/202002/07/2827498...,732,[{'avatar': 'https://di.phncdn.com/pics/users/...,[{'title': 'Sexy StepSisters Share StepBrother...,https://www.pornhub.com/view_video.php?viewkey...,NaN
3,"If you Wanna take a Bath, Fuck me first - Mini...","4,216,203",19,4,81,Mini Diva,849,"[Amateur, Babe, Big Ass, Big Tits, Blowjob, Ex...","[big boobs, teenager, young, bathroom sex, bat...",homemade,...,1179,2020-05-17T17:34:16.000Z,No Data,{'1080P': 'https://cv-h.phncdn.com/hls/videos/...,https://di.phncdn.com/videos/202005/17/3148084...,400,[{'avatar': 'https://di.phncdn.com/pics/users/...,[{'title': 'Romantic sex ended with huge cumsh...,https://www.pornhub.com/view_video.php?viewkey...,NaN
4,I'm sure he liked to Fuck the Whore he Paid fo...,"35,840,081",125,42,74,Mini Diva,849,"[Big Ass, Big Tits, Blowjob, Exclusive, Female...","[rough, butt, big boobs, teenager, young, poin...",homemade,...,867,2019-11-16T11:09:30.000Z,No Data,{'1080P': 'https://ev-h.phncdn.com/hls/videos/...,https://ei.phncdn.com/videos/201911/16/2620499...,911,[{'avatar': 'https://ci.phncdn.com/pics/users/...,[{'title': 'Busty college girl fucked with her...,https://www.pornhub.com/view_video.php?viewkey...,NaN


# Cleaning

## Clean comments function

In [ ]:
def isCleanMessage(text):

    # Detect bots
    if str(re.search(r'HTTP', text)) != "None":
        return False;

    if str(re.search(r'LIKE IF', text)) != "None":
        return False;

    if str(re.search(r'BROWSE', text)) != "None":
        return False;

    
    if str(re.search(r'MY PAGE', text)) != "None":
        return False;

    if str(re.search(r'MY CHANNEL', text)) != "None":
        return False;

    if str(re.search(r'SNAP', text)) != "None":
        return False;

    if str(re.search(r'MY PROFILE', text)) != "None":
        return False;
    
    if str(re.search(r'LISTEN UP SOLDIERS', text)) != "None":
        return False;

    # print("Clean message: ",text)
    return True


def sortComments(comments):
    clean = []
    dirty = []
    # print(co)
    for comment in comments:
        # tmp = comment['message'].upper() # if you want to keep the original message without uppercase but then need to make it uppercase
        # if (isCleanMessage(tmp])):
        comment['message'] = comment['message'].upper()
        if (isCleanMessage(comment['message'])):
            clean.append(comment)
        else:
            dirty.append(comment)

    return clean, dirty

def replaceWithCleanComments(row):
    clean, dirty = sortComments(row['comments'])
    # row['comments'] = []
    row['comments'] = {'clean': clean, 'dirty': dirty}
    # row['clean_comments'] = clean
    # row['clean_comments'] = []
    # row['dirty_comments'] = dirty

# Aggregate function
def cleanDataFrame(dataframe):
    for i, r in dataframe.iterrows():
        replaceWithCleanComments(r)

## Apply the function on the data

In [ ]:
cleanDataFrame(data)

# Save the file as Json

In [ ]:
data.to_json(r'CleanDataComments.json')
# data.to_json(r'CleanDataComments_TEST.json')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5b677b82-ea1e-4dc8-9b2a-3c037ad9d27c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>